# 1. 讀取檔案

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# 讀取資料
# 請確認這兩個 csv 檔案跟你的 ipynb 檔在同一個資料夾
try:
    df_train = pd.read_csv('data/train.csv')
    df_test = pd.read_csv('data/test.csv')
    print("成功讀取檔案！")
    print(f"訓練集大小: {df_train.shape}")
    print(f"測試集大小: {df_test.shape}")
except FileNotFoundError:
    print("找不到檔案！請確認檔名與路徑是否正確。")

# 2.資料預處理

In [ ]:
# 定義預處理函數 (方便同時處理訓練集與測試集)
def preprocess_data(data):
    df = data.copy()
    
    # 1. 處理 horsepower 的異常值與缺失值
    # 先強制轉為數字，無法轉的變 NaN
    df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
    
    # [優化] 聰明補值：根據 'cylinders' (汽缸數) 分組來補 horsepower 的中位數
    # 這樣補比補全體平均更準確 (8缸車補8缸的數值)
    df['horsepower'] = df['horsepower'].fillna(df.groupby('cylinders')['horsepower'].transform('median'))
    # 如果還有漏網之魚 (例如測試集有沒看過的汽缸數)，用全體中位數兜底
    df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

    # 2. 處理類別特徵 origin (One-Hot Encoding)
    # 產生 origin_usa, origin_japan, origin_europe
    df = pd.get_dummies(df, columns=['origin'], prefix='origin')
    
    return df

# 分別對訓練集和測試集進行預處理
df_train_clean = preprocess_data(df_train)
df_test_clean = preprocess_data(df_test)

print("資料預處理完成！")
df_train_clean.head()

# 3.特徵工程

In [ ]:
# [優化] 強力特徵工程 (Feature Engineering) - 加入物理意義特徵
def add_features(data):
    df = data.copy()
    # 馬力重量比 (Power-to-weight ratio)：越重越耗油，馬力越大通常越耗油
    df['power_weight_ratio'] = df['horsepower'] / df['weight']
    # 單缸排氣量：反映引擎構造
    df['disp_per_cyl'] = df['displacement'] / df['cylinders']
    # 加速度與馬力比
    df['acc_power_ratio'] = df['acceleration'] / df['horsepower']
    return df

df_train_fe = add_features(df_train_clean)
df_test_fe = add_features(df_test_clean)

# 定義要用來預測的特徵欄位
features = ['cylinders', 'displacement', 'horsepower', 'weight', 
            'acceleration', 'model_year', 
            'origin_usa', 'origin_japan', 'origin_europe',
            'power_weight_ratio', 'disp_per_cyl', 'acc_power_ratio']

target = 'mpg'

# 防呆：確保欄位一致 (如果測試集剛好缺某個產地，補 0)
for col in features:
    if col not in df_test_fe.columns:
        df_test_fe[col] = 0
    if col not in df_train_fe.columns:
        df_train_fe[col] = 0

print(f"使用的特徵 (共 {len(features)} 個): {features}")

X = df_train_fe[features]
y = df_train_fe[target]

# [關鍵優化] Target Log-Transform
# 對 mpg 取 log，讓資料分佈更接近常態，模型更好學
y_log = np.log1p(y)

# 4. 訓練模型

In [ ]:
# 分割訓練集與驗證集
X_train, X_val, y_train_log, y_val_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

# 使用 Gradient Boosting Regressor
# 參數經過微調 (High Baseline 設定)
model = GradientBoostingRegressor(n_estimators=1500,  # 增加樹的數量
                                  learning_rate=0.03, # 降低學習率，學得更細
                                  max_depth=4,        # 深度適中
                                  subsample=0.8,      # 隨機採樣防止過擬合
                                  random_state=42)

print("開始訓練模型...")
model.fit(X_train, y_train_log)

# 驗證模型表現
# 預測驗證集 (這時候預測出來的是 log mpg)
val_pred_log = model.predict(X_val)

# [重要] 把預測結果轉回原本的單位 (exp)
val_pred = np.expm1(val_pred_log)
y_val_original = np.expm1(y_val_log)

rmse = np.sqrt(mean_squared_error(y_val_original, val_pred))
print(f'驗證 RMSE: {rmse:.4f}')
# 如果 RMSE 低於 2.5，表示模型表現相當不錯

模型已訓練完成！
訓練誤差 (Train RMSE): 3.4119 MPG
測試誤差 (Test RMSE):  3.8504 MPG

--- 模型學到的關係 ---
特徵 'weight' 的權重: -0.0067
特徵 'acceleration' 的權重: 0.0325
特徵 'model_year' 的權重: 0.7999
特徵 'cylinders' 的權重: 0.3731
特徵 'displacement' 的權重: -0.0031
特徵 'horsepower' 的權重: -0.0095


# 5.輸出提交檔案

In [ ]:
# 對測試集進行預測
print("正在預測測試集...")
test_pred_log = model.predict(df_test_fe[features])

# 轉回原本的 scale
test_pred = np.expm1(test_pred_log)

# 建立提交 DataFrame
submission = pd.DataFrame({
    'id': df_test['id'], # 確保這裡是測試集的 id
    'mpg': test_pred
})

# 存檔
submission.to_csv('submission.csv', index=False)
print("submission.csv 已生成")
print(submission.head())

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：林宥臣 學號：111303547

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：2.57211  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：
        使用模型：Gradient Boosting Regressor
        缺失值處理：使用全體資料的「中位數」填補 horsepower。
        編碼處理：對 origin 進行 One-Hot Encoding (轉為 usa, japan, europe 欄位)。
    結果與觀察 (分數變化、心得等)：
        雖然使用了強大的梯度提升模型，但分數仍高於 Medium Baseline，表示模型還有改進空間。
        觀察發現，單純使用原始特徵（如重量、馬力）可能不足以讓模型抓到油耗的關鍵非線性關係，且 horsepower 的簡單補值可能造成了部分資料失真（例如 8 缸車補到了 4 缸車的數值）。  
    該次實驗分數： 2.87129

【實驗 2】  
    我做的修改：
        使用模型 (Model)：Gradient Boosting Regressor (參數微調：learning_rate=0.03, n_estimators=1500)
        分組補值：改用 cylinders 分組來填補 horsepower，大幅提升補值準確度。
        物理特徵工程：加入 power_weight_ratio (馬力重量比)、disp_per_cyl (單缸排氣量) 等具有物理意義的新特徵。
        目標變數轉換：對 mpg 進行 np.log1p (對數轉換)。  
    結果與觀察 (分數變化、心得等)：
        經過優化後，RMSE 大幅下降了約 0.3，成功突破 Medium Baseline。
        Log 轉換的效果最為顯著，它解決了油耗資料分佈不均的問題，讓模型對高油耗與低油耗車輛的預測都更加穩定。同時，加入馬力重量比等特徵後，模型能更直接地學習到車輛物理特性對油耗的影響。  
    該次實驗分數： 2.57211  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：
    本次作業主要集中在 Gradient Boosting 模型的優化過程。

    1. 從失敗中學習：在實驗 1 中，我原本以為只要換用 GBR 模型就能輕鬆過關，但結果卻不如預期。這讓我意識到模型只是工具，資料的品質與特徵的表達才是關鍵。

    2. 特徵工程的價值：在實驗 2 中，我不去盲目調整模型參數，而是回歸資料本質。透過加入「馬力重量比」這類領域知識特徵，以及修正補值策略（分組補值），模型立刻就有了顯著的進步。

    3. 數據分佈的處理：最大的突破點在於對 Target (mpg) 進行 Log 轉換。這驗證了在迴歸問題中，處理長尾分佈對於提升 RMSE 有著決定性的影響。最終以 2.57 的成績順利完成目標。